In [9]:
from google.colab import drive
drive.mount('/content/drive')

# Liste as pastas logo abaixo de MyDrive
!ls -lha "/content/drive/MyDrive"

# Procure onde estão seus arquivos .json do LabelMe
!find "/content/drive/MyDrive" -iname "*.json" | head -n 50


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 8.4G
-rw------- 1 root root  11K Nov 29  2022 '2200 - Aula 2 - Modelo de Briefing de SEO.xlsx'
-rw------- 1 root root 286K Apr  6 18:38  321581403
-rw------- 1 root root 334K Apr  6 18:38 '321581403 (1)'
-rw------- 1 root root 286K Apr  6 18:37 '321581403 (2)'
-rw------- 1 root root 1.1M Dec  8  2021  350D8457-9C9E-4B7F-92D2-CCCC15B959D2.jpeg
-rw------- 1 root root 432K Aug 14  2020  8.PNG
-rw------- 1 root root 4.9K Jul 17  2021  aceleradoradesafios.zip
-rw------- 1 root root  177 Apr 11  2024 'adriel codigo os.gsheet'
-rw------- 1 root root 649K Jul 13  2021  AE7728AD-9709-425C-8F5C-B3B844DA13D5.jpeg
-rw------- 1 root root  177 Apr  6  2018 'alimentos terminar.gdoc'
-rw------- 1 root root  177 Nov 18  2021 'alura dashboard.gsheet'
-rw------- 1 root root 147K Oct 12  2019 'A Mediação como ferramenta de auxílio para a Governança Empresarial.doc'
-r-

In [11]:
# === LabelMe (JSON) -> YOLO (txt) com split train/val + data.yaml ===
import json, shutil, random
from pathlib import Path
from PIL import Image

RAW_LABELME = Path("/content/drive/MyDrive/SEU/CAMINHO/AQUI")  # <-- TROQUE aqui!
OUT_YOLO    = Path("/content/dataset_yolo")
VAL_SPLIT   = 0.2
SEED        = 1337

assert RAW_LABELME.exists(), f"Pasta não encontrada: {RAW_LABELME}"
json_files = sorted(RAW_LABELME.rglob("*.json"))
assert json_files, f"Nenhum .json em {RAW_LABELME}"

labels_set, samples = set(), []
def find_image(jpath, imagePath):
    p = jpath.parent / imagePath
    if p.exists(): return p
    stem = Path(imagePath).stem
    for ext in (".jpg",".jpeg",".png",".bmp"):
        cand = jpath.parent / f"{stem}{ext}"
        if cand.exists(): return cand
    return None

for jf in json_files:
    data = json.loads(jf.read_text())
    img = find_image(jf, data.get("imagePath",""))
    if not img:
        print(f"[AVISO] sem imagem para {jf.name}");
        continue
    shapes = data.get("shapes", [])
    labels_set.update(s.get("label") for s in shapes if s.get("label"))
    samples.append((img, shapes, data.get("imageWidth"), data.get("imageHeight")))

CLASSES = sorted([c for c in labels_set if c])
assert CLASSES, "Nenhuma classe encontrada nos JSONs."
cls2id = {c:i for i,c in enumerate(CLASSES)}
print("Classes detectadas:", CLASSES)

for d in ("images/train","images/val","labels/train","labels/val","tmp/images","tmp/labels"):
    (OUT_YOLO/d).mkdir(parents=True, exist_ok=True)

def bbox_from_points(points):
    xs = [pt[0] for pt in points]
    ys = [pt[1] for pt in points]
    x1, y1, x2, y2 = min(xs), min(ys), max(xs), max(ys)
    return x1, y1, x2, y2

def yolo_line(id_, box, W, H):
    x1,y1,x2,y2 = box
    cx = ((x1+x2)/2)/W
    cy = ((y1+y2)/2)/H
    w  = (x2-x1)/W
    h  = (y2-y1)/H
    return f"{id_} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}"

kept = []
for img_path, shapes, W, H in samples:
    if not W or not H:
        with Image.open(img_path) as im: W,H = im.size
    dst_img = OUT_YOLO/'tmp/images'/f"{img_path.stem}{img_path.suffix.lower()}"
    if not dst_img.exists():
        shutil.copy2(img_path, dst_img)
    lines = []
    for s in shapes:
        lbl = s.get("label")
        if lbl not in cls2id: continue
        pts = s.get("points", [])
        if len(pts) < 2: continue
        x1,y1,x2,y2 = bbox_from_points(pts)
        if x2<=x1 or y2<=y1: continue
        lines.append(yolo_line(cls2id[lbl], (x1,y1,x2,y2), W, H))
    (OUT_YOLO/'tmp/labels'/f"{img_path.stem}.txt").write_text("\n".join(lines))
    kept.append(img_path.stem)

random.seed(SEED); random.shuffle(kept)
n_val = max(1, int(len(kept)*VAL_SPLIT))
val_set = set(kept[:n_val])

def move_pair(stem, split):
    src_img = list((OUT_YOLO/'tmp/images').glob(stem + ".*"))[0]
    src_lbl = OUT_YOLO/'tmp/labels'/f"{stem}.txt"
    (OUT_YOLO/f"images/{split}").mkdir(parents=True, exist_ok=True)
    (OUT_YOLO/f"labels/{split}").mkdir(parents=True, exist_ok=True)
    shutil.copy2(src_img, OUT_YOLO/f"images/{split}"/src_img.name)
    shutil.copy2(src_lbl, OUT_YOLO/f"labels/{split}"/src_lbl.name)

for stem in kept:
    move_pair(stem, "val" if stem in val_set else "train")

shutil.rmtree(OUT_YOLO/'tmp', ignore_errors=True)
(OUT_YOLO/'classes.txt').write_text("\n".join(CLASSES))
(OUT_YOLO/'data.yaml').write_text(
    f"path: {OUT_YOLO}\ntrain: images/train\nval: images/val\nnames: {CLASSES}\n"
)

print("✅ Conversão concluída!")
print("Data.yaml:\n", (OUT_YOLO/'data.yaml').read_text())


AssertionError: Pasta não encontrada: /content/drive/MyDrive/SEU/CAMINHO/AQUI

In [12]:
!pip -q install ultralytics
from ultralytics import YOLO

DATA_YAML = "/content/dataset_yolo/data.yaml"
model = YOLO("yolov8n.pt")
model.train(data=DATA_YAML, epochs=20, imgsz=640, batch=16, patience=5)


Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset_yolo/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=5, perspective=0.0, plots=True, po

RuntimeError: Dataset '/content/dataset_yolo/data.yaml' error ❌ '/content/dataset_yolo/data.yaml' does not exist

In [13]:
!ls -R "/content/dataset_yolo"
!cat "/content/dataset_yolo/data.yaml"


ls: cannot access '/content/dataset_yolo': No such file or directory
cat: /content/dataset_yolo/data.yaml: No such file or directory


In [ ]:
from ultralytics import YOLO
YOLO("yolov8n.pt").train(data="coco128.yaml", epochs=1, imgsz=640)


Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco128.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretraine